In [1]:
import argparse
import glob
from pathlib import Path

try:
    import open3d
    from visual_utils import open3d_vis_utils as V
    OPEN3D_FLAG = True
except:
    import mayavi.mlab as mlab
    from visual_utils import visualize_utils as V
    OPEN3D_FLAG = False

import numpy as np
import torch
import time
import demo as demo

from pcdet.config import cfg, cfg_from_yaml_file
from pcdet.datasets import DatasetTemplate
from pcdet.models import build_network, load_data_to_gpu
from pcdet.utils import common_utils

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
ckpt_path = '../self_ckpts/pointpillar_7728.pth'
cfg_path = './cfgs/kitti_models/pointpillar.yaml'
data_path = '/home/ghosnp/dataset/mini_kitti/velodyne/testing/velodyne/000003.bin'

logger = common_utils.create_logger()   # logger.info('xxx')
logger.info('Test to get the VFE params' + time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
cfg_from_yaml_file(cfg_path, cfg)


demo_dataset = demo.DemoDataset(
    dataset_cfg=cfg.DATA_CONFIG, class_names=cfg.CLASS_NAMES, training=False,
    root_path=Path(data_path), ext='.bin', logger=logger
)

model = build_network(model_cfg=cfg.MODEL, num_class=len(cfg.CLASS_NAMES), dataset=demo_dataset)
model.load_params_from_file(filename=ckpt_path, logger=logger, to_cpu=True)

2023-12-06 16:52:49,196   INFO  Test to get the VFE params2023-12-06 16:52:49


PillarVFE==================
4 [0.16, 0.16, 4] [  0.   -39.68  -3.    69.12  39.68   1.  ]
PFN==================
10 64 True True
SCATTER==========================
[432 496   1]


/home/ghosnp/miniconda3/envs/pcdet/lib/python3.8/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
2023-12-06 16:52:51,954   INFO  ==> Loading parameters from checkpoint ../self_ckpts/pointpillar_7728.pth to CPU
2023-12-06 16:52:51,968   INFO  ==> Done (loaded 127/127)


In [3]:
# vfe.pfn_layers.0.linear.weight torch.Size([64, 10])
# vfe.pfn_layers.0.norm.weight torch.Size([64])
# vfe.pfn_layers.0.norm.bias torch.Size([64])

# get the vfe params
# vfe_pfn_weight = model.named_parameters().__next__()[1].detach().cpu().numpy()


for name, param in model.named_parameters():
    print(name, param.shape)
    if name == 'vfe.pfn_layers.0.linear.weight':
        vfe_pfn_weight = param.detach().cpu().numpy()
    elif name == 'vfe.pfn_layers.0.norm.weight':
        vfe_pfn_norm_weight = param.detach().cpu().numpy()
    elif name == 'vfe.pfn_layers.0.norm.bias':
        vfe_pfn_norm_bias = param.detach().cpu().numpy()
    else:
        pass

print(vfe_pfn_weight.shape, vfe_pfn_norm_weight.shape, vfe_pfn_norm_bias.shape)


vfe.pfn_layers.0.linear.weight torch.Size([64, 10])
vfe.pfn_layers.0.norm.weight torch.Size([64])
vfe.pfn_layers.0.norm.bias torch.Size([64])
backbone_2d.blocks.0.1.weight torch.Size([64, 64, 3, 3])
backbone_2d.blocks.0.2.weight torch.Size([64])
backbone_2d.blocks.0.2.bias torch.Size([64])
backbone_2d.blocks.0.4.weight torch.Size([64, 64, 3, 3])
backbone_2d.blocks.0.5.weight torch.Size([64])
backbone_2d.blocks.0.5.bias torch.Size([64])
backbone_2d.blocks.0.7.weight torch.Size([64, 64, 3, 3])
backbone_2d.blocks.0.8.weight torch.Size([64])
backbone_2d.blocks.0.8.bias torch.Size([64])
backbone_2d.blocks.0.10.weight torch.Size([64, 64, 3, 3])
backbone_2d.blocks.0.11.weight torch.Size([64])
backbone_2d.blocks.0.11.bias torch.Size([64])
backbone_2d.blocks.1.1.weight torch.Size([128, 64, 3, 3])
backbone_2d.blocks.1.2.weight torch.Size([128])
backbone_2d.blocks.1.2.bias torch.Size([128])
backbone_2d.blocks.1.4.weight torch.Size([128, 128, 3, 3])
backbone_2d.blocks.1.5.weight torch.Size([128])


In [11]:
from pcdet.models.backbones_3d.vfe import pillar_vfe as VFE


# points
# frame_id
# lidar_aug_matrix
# use_lead_xyz
# voxels
# voxel_coords
# voxel_num_points
# batch_size
# pillar_features
# spatial_features
# spatial_features_2d
# batch_cls_preds
# batch_box_preds
# cls_preds_normalized


model.cuda()
model.eval()

with torch.no_grad():
    for idx, data_dict in enumerate(demo_dataset):
        data_dict = demo_dataset.collate_batch([data_dict])
        load_data_to_gpu(data_dict)
        pred_dicts, _ = model.forward(data_dict)
        print(idx,data_dict['pillar_features'].shape)

print('INFOS')
print(data_dict['spatial_features'].shape)
print(data_dict['spatial_features_2d'].shape)

print(pred_dicts)

PILLAR FEATURES=====================
torch.Size([8494, 64])
SPATIAL FEATURES=====================
torch.Size([1, 64, 496, 432])
0 torch.Size([8494, 64])
INFOS
torch.Size([1, 64, 496, 432])
torch.Size([1, 384, 248, 216])
[{'pred_boxes': tensor([[16.9990,  3.7984, -0.9429,  4.4218,  1.6759,  1.4718,  6.2595],
        [ 5.3099, 11.3921, -1.2433,  3.5853,  1.5457,  1.4378,  3.1575],
        [30.8133, -0.5729, -0.7571,  4.3241,  1.6616,  1.5736,  6.2604],
        [53.2946,  2.0208, -0.4022,  3.9298,  1.6297,  1.5902,  6.1712],
        [53.3183, -2.7357, -0.4548,  3.8728,  1.5616,  1.6064,  6.0431],
        [18.7330, 15.5931, -1.3265,  4.4662,  1.6952,  1.5360,  6.3886],
        [ 0.6563, -0.5786, -1.1316,  4.1750,  1.6669,  1.5610,  6.0350],
        [64.3181,  8.4915, -0.4889,  4.0612,  1.5960,  1.5436,  6.2763]],
       device='cuda:0'), 'pred_scores': tensor([0.9588, 0.9466, 0.9179, 0.5074, 0.4411, 0.3232, 0.1731, 0.1461],
       device='cuda:0'), 'pred_labels': tensor([1, 1, 1, 1, 1, 1, 